In [1]:
# add path to sispeuede to sys.path in python
import sys
import warnings
dir_git = "/Users/usuario/git/sisepuede"
if dir_git not in sys.path:
    sys.path.append(dir_git)


warnings.filterwarnings("ignore")

import logging
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import pandas as pd
import pathlib
import sisepuede as ssp
import sisepuede.core.model_attributes as ma
import sisepuede.core.model_variable as mv
import sisepuede.core.support_classes as sc
import sisepuede.manager.sisepuede_examples as sxm
import sisepuede.manager.sisepuede_file_structure as sfs
import sisepuede.manager.sisepuede_models as sm
import sisepuede.transformers as trf
import sisepuede.utilities._toolbox as sf
import time
from typing import *




def _setup_logger(namespace: str, fn_out: Union[str, None] = None) -> None:
    global logger
    
    format_str = "%(asctime)s - %(levelname)s - %(message)s"
    # configure
    if fn_out is not None:
        logging.basicConfig(
            filename = fn_out,
            filemode = "w",
            format = format_str,
            level = logging.DEBUG
        )
    else:
        logging.basicConfig(
            format = format_str,
            level = logging.DEBUG
        )
        
    logger = logging.getLogger(namespace)
    # create console handler and set level to debug
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter
    formatter = logging.Formatter(format_str)
    # add formatter to ch
    ch.setFormatter(formatter)
    # add ch to logger
    logger.addHandler(ch)

    return logger

_setup_logger(__name__, os.path.join(os.getcwd(), "log_build_sisepuede_transformations.log"))


#
#   WRITE EVERYTHING TO OUTPUTS
#

# write outputs
dir_out = "/Users/usuario/Documents/Personal Files/Other Work/Louisiana/temp_data"
if not os.path.exists(dir_out):
    os.makedirs(dir_out, exist_ok = True)


# Setup the file structure and read in data

In [2]:
struct = sfs.SISEPUEDEFileStructure()
examples = sxm.SISEPUEDEExamples()

matt = struct.model_attributes
regions = sc.Regions(matt)
time_periods = sc.TimePeriods(matt)

# directories
path_cur = pathlib.Path(os.getcwd())
path_data = path_cur.joinpath("data")
path_transformations = path_cur.joinpath("transformations")
if not path_transformations.exists():
    path_transformations.mkdir(exist_ok = True, )

# file paths
fp_inputs = "/Users/usuario/Documents/Projects/Louisiana/quick_data/louisiana_working_data_20240826/MODEL_INPUTS.csv"



##  Format input data

In [3]:
# read in default data
regions_run = ["louisiana"]

df_inputs_base_la = pd.read_csv(fp_inputs, )
df_input = df_inputs_base_la.copy()


# overwrite some variables
for mvname in ["Agriculture Equivalent Exports", "Livestock Equivalent Exports"]:
    modvar = matt.get_variable(mvname)
    if modvar is None:
        raise RuntimeError(f"Error: model variable '{mvname}' not found.")
    df_input[modvar.fields] = 0


# filter
field_region = regions.key
regions_run = [
    x for x in regions_run if x in list(df_input[field_region].unique())
]

# reduce inputs
df_input = (
    df_input[
        df_input[field_region].isin(regions_run)
    ]
    .drop(columns = [matt.dim_primary_id])
    .reset_index(drop = True)
)


# build transformations

In [5]:
transformers = trf.Transformers(
    {},
    df_input = df_input,
)

if not path_transformations.exists():
    
    print("BUILDING DEFAULT TRANSFORMATIONS")
    
    trf.instantiate_default_strategy_directory(
        transformers,
        path_transformations,
    )


# then, you can load this back in after modifying (play around with it)
transformations = trf.Transformations(
    path_transformations,
    transformers = transformers,
)
tab = transformations.attribute_transformation.table


# build strategies
#  build the strategies -- will export to path
t0 = time.time()
strategies = trf.Strategies(
    transformations,
    export_path = "transformations",
    prebuild = True,
)

t_elapse = sf.get_time_elapsed(t0)
print(f"Strategies defined at {strategies.transformations.dir_init} initialized in {t_elapse} seconds")

# build whirlpool with existing data
# tss = strategies.build_whirlpool_strategies(6004, ids = list(range(7101, 7160)))





Strategies defined at /Users/usuario/git/sisepuede_region_nbs/louisiana/transformations initialized in 1.6 seconds


In [6]:
df_vargroups = examples("variable_trajectory_group_specification")
strategies.build_strategies_to_templates(
    df_trajgroup = df_vargroups,
    include_simplex_group_as_trajgroup = True,
    strategies = [0, 6002],#strategies.all_strategies,#list(range(7101, 7160)),#[0, 6003, 6004, 6005] #+ list(range(7001, 7062)), # list(range(7101, 7160))
)


0